# Slamseq Ibet re-analysis from Muhar Paper

In [ ]:
from __future__ import print_function
import os.path
import pandas as pd
import sys
sys.path.insert(0, '../../')
import seaborn as sns
import numpy as np

from JKBio.Helper import *
from JKBio.helper import pyDESeq2

from bokeh.plotting import *
from bokeh.models import HoverTool
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
#from umap import UMAP

output_notebook()
%load_ext autoreload
%matplotlib inline
%autoreload 2
%load_ext rpy2.ipython

downloading their data

## downloading the data

In [ ]:
version="v4"
project="Muhar_Slamseq"
location= '../data/'+project+"/"

In [ ]:
gsutil -m cp -r gsutil ls gs://amlproject/RNA/slamseq/MYCpaper/ ../../data/$project/

In [ ]:
location= '../../data/'+project+'/count/'

In [ ]:
files = ! ls $location*.tsv
files

In [ ]:
files = files[:6] +[files[i] for i in [-8,-6,-4]] + files[-3:]

## Running slamdunk

In [ ]:
#if need to download the data from sra
! mkdir ../../data/slam/MYCpaper && cd ../../data/slam/MYCpaper && fastq_dump SRR5806781 && fastq_dump SRR5806783 && fastq_dump SRR5806785 &&
fastq_dump SRR5806780 && fastq_dump SRR5806782 && fastq_dump SRR5806784 && cd -

In [ ]:
folder_bams = "../../data/slam/MYCpaper/"
bams = ! ls $folder_bams
bams

In [ ]:
! pip3 install git+https://github.com/jkobject/slamdunk.git --upgrade

In [ ]:
for val in bams[1:]:
    print(val)
    os.system('../../TrimGalore-0.6.5/trim_galore -stringency 3 \
    --illumina '+folder_bams+val + ' -o '+folder_bams) 

In [ ]:
ls -al ../../data/slam/res/count

In [ ]:
for val in bams[1:]:
    print(val)
    os.system('slamdunk all -r ../../data/slam/ref_genome.fa -b ../../data/slam/GSE100708_hg38_refseq_062016_ensemblv84_3UTR.bed \
    -o ../../data/slam/res/ -t 8 -c 2 '+folder_bams+val.split('.')[0]+'_trimmed.fq')

In [ ]:
ls -al ../../data/slam/res/filter

In [ ]:
for val in bams[1:]:
    print(val)
    os.system('slamdunk count -r ../../data/slam/ref_genome.fa \
    -b ../../data/slam/GSE100708_hg38_refseq_062016_ensemblv84_3UTR.bed \
    -o ../../data/slam/res/count/ -t 8 -c 2 -s ../../data/slam/res/snp \
    ../../data/slam/res/filter/'+ val.split('.')[0]+"_trimmed_slamdunk_mapped_filtered.bam")

## we first need to recreate out RNA matrix from slamdunk's tcount files

In [ ]:
data={}
for file in files:
    data[file.split('.')[-3].split('/')[-1]]= pd.read_csv(file, sep='\t', comment='#',header=0)

In [ ]:
prev = -2
for k,val in data.items():
    if len(set(val.Name)) != prev and prev!=-2: 
        raise ValueError('we do not have the same number of genes in each file')
    prev = len(set(val.Name))

![](images/conversionrates.png)

In [ ]:
data.keys()

In [ ]:
val

In [ ]:
flag_var= 100

readcounts = {i:[0]*len(data) for i in val.Name.unique()}
tccounts = {i:[0]*len(data) for i in val.Name.unique()}

In [ ]:
for n, (_,val) in enumerate(data.items()):
    val = val.sort_values(by="Name")
    j  = 0
    #print('              ',end='\r')
    readcount = [val.iloc[0].ReadCount]
    tccount = [val.iloc[0].TcReadCount]
    prevname = val.iloc[0].Name
    for _, v in val.iloc[1:].iterrows():
        if v.Name==4609:
            print(v.ReadCount,v.TcReadCount)
            print(readcount,tccount)
        if v.Name == prevname:
            readcount.append(v.ReadCount)
            tccount.append(v.TcReadCount)
        else:
            readcounts[prevname][n] = np.sum(readcount)
            tccounts[prevname][n] = np.sum(tccount)
            #if np.var(readcount) > flag_var:
            #    print("pb with "+str(v.Name))
            prevname=v.Name
            j+=1
            #print(j,end='\r')
            readcount = [v.ReadCount]
            tccount = [v.TcReadCount]
files = [*data]

In [ ]:
val.Name

In [ ]:
readcounts = pd.DataFrame(data=readcounts,columns=val.Name.unique(),index=data.keys()).T
tccounts = pd.DataFrame(data=tccounts,columns=val.Name.unique(),index=data.keys()).T

In [ ]:
names, _ = convertGenes(readcounts.index.tolist(), from_idtype="entrez_id", to_idtype="symbol")
readcounts.index = names

In [ ]:
names, _ = convertGenes(tccounts.index.tolist(), from_idtype="entrez_id", to_idtype="symbol")
tccounts.index = names

In [ ]:
tccounts.values.mean()

In [ ]:
minvar_toremove = 2
mincount_toremove = 10

In [ ]:
nottodrop = np.argwhere(tccounts.values.var(0)>=lowvar_toremove).ravel()
#tccounts = tccounts.iloc[nottodrop]
#readcounts = readcounts.iloc[nottodrop]

In [ ]:
nottodrop=np.argwhere(readcounts.values.max(1)>=mincount_toremove).ravel()
tccounts = tccounts.iloc[nottodrop]
readcounts = readcounts.iloc[nottodrop]

In [ ]:
len(readcounts)

## SLAM-seq
All SLAM-seq assays were performed at 60-70% confluency for adherent cells or
60% of the maximum cell density counted on a hemocytometer for suspension cells. 5-7h
prior to each assay, growth medium was aspirated and replaced. Unless stated otherwise,
cells were pre-treated with indicated small molecule inhibitors or 100µM IAA for 30 min
to pre-establish full target inhibition or degradation. Newly synthesized RNA was labeled
for indicated time spans (45 min or 60 min) at a final concentration of 100µM 4-
thiouridine (4sU, Carbosynth). Adherent cells were harvested by direct snap-freezing of
plates on dry ice. Suspension cells were spun down and immediately snap-frozen. RNA
extraction was performed using the RNeasy Plus Mini Kit (Qiagen). 

Total RNA was subjected to alkylation by iodoacetamide (Sigma, 10mM) for 15 min and RNA was repurified by ethanol precipitation. 500ng alkylated RNA were used as input for generating
3’-end mRNA sequencing libraries using a commercially available kit (QuantSeq 3′
mRNA-Seq Library Prep Kit FWD for Illumina and PCR Add-on Kit for Illumina,
Lexogen). Deep sequencing was performed using HiSeq1500 and HiSeq2500 platforms
(Illumina).

0.06724463
0.04916348
0.03728357
0.05075007
0.03331248
0.0346662
0.03265504 
0.04416262
0.05574821
0.05845953
0.05531433
0.06175393
0.03181749 
0.02940226 
0.0306263
0.04004278

In [ ]:
names = {
"GSM2691910": "K562_MYC-AID_DMSO_rep-1",
"GSM2691911": "K562_MYC-AID_DMSO_rep-2",
"GSM2691912": "K562_MYC-AID_DMSO_rep-3",
"GSM2691913": "K562_MYC-AID_IAA_rep-1",
"GSM2691914": "K562_MYC-AID_IAA_rep-2",
"GSM2691915": "K562_MYC-AID_IAA_rep-3",
"GSM2691916": "HCT116_MYC-AID_DMSO_rep-1",
"GSM2691917": "HCT116_MYC-AID_DMSO_rep-2",
"GSM2691918": "HCT116_MYC-AID_DMSO_rep-3",
"GSM2691919": "HCT116_MYC-AID_IAA_rep-1",
"GSM2691920": "HCT116_MYC-AID_IAA_rep-2",
"GSM2691921": "HCT116_MYC-AID_IAA_rep-3",
"GSM2691882": "MOLM-13_DMSO_rep-1_exp-2",
"GSM2691883": "MOLM-13_DMSO_rep-2_exp-2",
"GSM2691884": "MOLM-13_DMSO_rep-3_exp-2",
"GSM2691885": "MOLM-13_JQ1_rep-1_exp-2",
"GSM2691886": "MOLM-13_JQ1_rep-2_exp-2",
"GSM2691887": "MOLM-13_JQ1_rep-3_exp-2",
"GSM2691888": "MOLM-13_6nM_NVP-2_rep-1_exp-2",
"GSM2691889": "MOLM-13_6nM_NVP-2_rep-2_exp-2",
"GSM2691890": "MOLM-13_6nM_NVP-2_rep-3_exp-2",
"GSM2691891": "MOLM-13_60nM_NVP-2_rep-1_exp-2",
"GSM2691892": "MOLM-13_60nM_NVP-2_rep-2_exp-2",
"GSM2691893": "MOLM-13_JQ1+6nM_NVP-2_rep-1_exp-2",
"GSM2691894": "MOLM-13_JQ1+6nM_NVP-2_rep-2_exp-2",
"GSM2691895": "MOLM-13_JQ1+6nM_NVP-2_rep-3_exp-2"
}

In [ ]:
tccounts = tccounts.rename(columns=names)
readcounts = readcounts.rename(columns=names)
readcounts

In [ ]:
pd.DataFrame(tccounts.values[:,:-1].astype(float)/readcounts.mean().values*100, columns=tccounts.columns[:-1], index=tccounts['genes']).to_csv('../results/'+project+'/'version+'_'+minvar_toremove+'_'+mincount_toremove+'_ratio.csv')

In [ ]:
(tccounts.values[:,:-1].astype(float)/(1+readcounts.values[:,:-1].astype(float))).mean(0)

In [ ]:
ratio.mean()

In [ ]:
tccounts.mean()

In [ ]:
readcounts.to_csv('../results/'+project+'/'version+'_'+minvar_toremove+'_'+mincount_toremove+'_readcounts.csv',index=False)

In [ ]:
tccounts.to_csv('../results/'+project+'/'version+'_'+minvar_toremove+'_'+mincount_toremove+'_tccounts.csv',index=False)

### make more dataframes

In [ ]:
mtccounts = pd.DataFrame()
mreadcounts = pd.DataFrame()
for i in range(4):
    mtccounts[tccounts.columns[i*3].split("_rep")[0]] = tccounts[tccounts.columns[(i*3):(i*3)+3]].mean(1)
    mreadcounts[tccounts.columns[i*3].split("_rep")[0]] = readcounts[readcounts.columns[(i*3):(i*3)+3]].mean(1)

In [ ]:
mtccounts.to_csv("../results/"+project+'/'version+'_'+minvar_toremove+'_'+mincount_toremove+"_mean_tccounts.csv")
mreadcounts.to_csv("../results/"+project+'/'version+'_'+minvar_toremove+'_'+mincount_toremove+"_mean_readcounts.csv")

In [ ]:
mtccounts = pd.DataFrame()
mreadcounts = pd.DataFrame()
for i in range(4):
    mtccounts[tccounts.columns[i*3].split("_rep")[0]] = tccounts[tccounts.columns[(i*3):(i*3)+3]].median(1)
    mreadcounts[tccounts.columns[i*3].split("_rep")[0]] = readcounts[readcounts.columns[(i*3):(i*3)+3]].median(1)

In [ ]:
mtccounts.to_csv("../results/"+project+'/'version+'_'+minvar_toremove+'_'+mincount_toremove+"_median_tccounts.csv")
mreadcounts.to_csv("../results/"+project+'/'version+'_'+minvar_toremove+'_'+mincount_toremove+"_median_readcounts.csv")

In [ ]:
(tccounts/readcounts).fillna(0).to_csv("../results/"+project+'/'version+'_'+minvar_toremove+'_'+mincount_toremove+"_tccounts_per_readcounts.csv")

# lets look at the similarity accross replicates

we have some outliers, else it seems to make some sense and on average, to be extremelly similar!

In [ ]:
counts.loc['MYC']

In [ ]:
tccounts.columns = [i.replace('-','.') for i in tccounts.columns]
readcounts.columns = [i.replace('-','.') for i in readcounts.columns]

In [ ]:
%matplotlib inline
sort = [0,8,10,4,6,14,12,1,11,9,5,7,13,3,15,2]
sns.heatmap(tccounts[tccounts.columns[sort]].corr(), 
            xticklabels=tccounts.columns[sort].values,
            yticklabels=tccounts.columns[sort].values)
plt.savefig('../results/'+project+'/plots/'+version+'_'+minvar_toremove+'_'+mincount_toremove+'_similarity_replicates_sorted_readcounts.csv')

In [ ]:
%matplotlib inline
sns.heatmap(readcounts.corr(), 
            xticklabels=readcounts.columns,
            yticklabels=readcounts.columns)
plt.savefig('../results/'+project+'/plots/'+version+'_similarity_replicates.csv')

In [ ]:
%matplotlib inline
sort = [0,8,10,4,12,6,14,1,9,5,13,7,11,15,3,2]
sns.heatmap(readcounts[readcounts.columns[sort]].corr(), 
            xticklabels=readcounts.columns.values[sort],
            yticklabels=readcounts.columns.values[sort])
plt.savefig('../results/'+project+'/plots/'+version+'_'+minvar_toremove+'_'+mincount_toremove+'_similarity_replicates_sorted_tccounts.csv')

read the paper for MV411 QC steps.

### A quick detour to look for clustering

## Getting the Core TF information

In [ ]:
ctf=pd.read_csv('../data/CTF.csv',header=None)[0].values.tolist()
ctf

In [ ]:
set(ctf) & set(tccounts.index)

In [ ]:
ctfpos = [val for val in tccounts.index if val in ctf]
notctfpos = [val for val in tccounts.index if val not in ctf]

In [ ]:
ctfpos.extend(['EGR1','SERTAD1'])

We find a CTF not in the dataset

# Let's look at how they change accross time

how core transcription factors change accross time when the cell is with JQ1 conditions

we are looking at the difference between production of each CTF compared to the average RNA production with JQ1 and without JQ1

## DESEQ ANALYSIS

In [ ]:
readcounts["genes"] = readcounts.index
readcounts = readcounts.reset_index(drop=True)
tccounts["genes"] = tccounts.index
tccounts = tccounts.reset_index(drop=True)

In [ ]:
tccounts.iloc[140]

In [ ]:
tccounts.columns

In [ ]:
design10um = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['VHL','DMSO'], 
                      data=np.array([[1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0],
                                     [0,0,0,0,1,1,1,0,0,0,0,0,0,1,0,0]],dtype=bool).T)
design1um = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['DMSO','VHL'], 
                      data=np.array([[0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0],
                                     [0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1]],dtype=bool).T)
design5um = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['DMSO','VHL'], 
                      data=np.array([[1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
                                     [0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0]],dtype=bool).T)

In [ ]:
designK562 = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['DMSO','JQ1'], 
                      data=np.array([[1,1,1,0,0,0,0,0,0,0,0,0],
                                     [0,0,0,1,1,1,0,0,0,0,0,0]],dtype=bool).T)
designMV411 = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['DMSO','JQ1'], 
                      data=np.array([[0,0,0,0,0,0,1,1,1,0,0,0],
                                     [0,0,0,0,0,0,0,0,0,1,1,1]],dtype=bool).T)

In [ ]:
design_HCT116 = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['DMSO','VHL'], 
                      data=np.array([[0,0,0,0,0,0,1,1,1,0,0,0],
                                     [0,0,0,0,0,0,0,0,0,1,1,1]],dtype=bool).T)
design_K562 = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['DMSO','VHL'], 
                      data=np.array([[1,1,1,0,0,0,0,0,0,0,0,0],
                                     [0,0,0,1,1,1,0,0,0,0,0,0]],dtype=bool).T)

In [ ]:
design1 = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['DMSO','JQ1'], 
                      data=np.array([[1,1,1,0,0,0,0,0,0,0,0,0,0,0],
                                     [0,0,0,1,1,1,0,0,0,0,0,0,0,0]],dtype=bool).T)
design2 = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['DMSO','NVP2low'], 
                      data=np.array([[1,1,1,0,0,0,0,0,0,0,0,0,0,0],
                                     [0,0,0,0,0,0,1,1,1,0,0,0,0,0]],dtype=bool).T)
design3 = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['DMSO','NVP2high'], 
                      data=np.array([[1,1,1,0,0,0,0,0,0,0,0,0,0,0],
                                     [0,0,0,0,0,0,0,0,0,1,1,0,0,0]],dtype=bool).T)
design4 = pd.DataFrame(index=[i.replace('-','.') for i in tccounts.columns.values[:-1]], 
                      columns=['DMSO','NVP2lowJQ1'], 
                      data=np.array([[1,1,1,0,0,0,0,0,0,0,0,0,0,0],
                                     [0,0,0,0,0,0,0,0,0,0,0,1,1,1]],dtype=bool).T)

### Differential gene expression analysis, PCA and GO-term enrichment

For gene-level analysis, raw reads mapped to different UTR annotations of the same gene were summed up by Entrez Gene ID. Pilot studies of K562 cells with kinase inhibitors were performed as single experiments. 

Analysis of differential gene expression was restricted to genes with ≥ 10 reads in at least one condition for 50bp sequencing runs (flavopiridol and DMSO) or ≥ 20 reads in at least one condition for 100bp sequencing runs (mk2206, trametinib, nilotinib, trametinib + mk2206 and DMSO). For estimating differential expression, a pseudo-count of 1 raw read was added to all genes.

#### All other SLAM-seq experiments were performed in triplicates and analyzed as follows. 
Differential gene expression calling was performed on raw read counts with ≥ 2 T>C conversions using DESeq2 (version 1.14.1) with default settings, and with size factors estimated on corresponding total mRNA reads for global normalization.

Downstream analysis was restricted to genes passing all internal filters for FDR estimation by DESeq2. Principal component analysis was performed after variance
stabilizing transformation on the 500 most variable genes across all conditions of a given
experiment. GO-term enrichment analysis was performed on genes significantly and strongly downregulated (FDR ≤ 0.1, log2FC ≤ -1) in SLAM-seq upon IAA-treatment in K562MYC-AID + Tir1 by the PANTHER Overrepresentation Test (Fisher's Exact with FDR multiple test correction, release 20171205, http://pantherdb.org) on GO Ontology database Released 2017-12-27.

In [ ]:
design

In [ ]:
tccounts.columns = [i.replace('-','.') for i in tccounts.columns]

In [ ]:
tccounts[tccounts.genes=='MYC']

In [ ]:
deseq10um = pyDESeq2.pyDESeq2(count_matrix=tccounts[tccounts.columns[np.array([1,0,0,0,1,1,1,0,1,1,1,0,0,1,0,0,1],np.bool)]], design_matrix=design[np.array([1,0,0,0,1,1,1,0,1,1,1,0,0,1,0,0],np.bool)],
                         design_formula="~DMSO - VHL",
                         gene_column="genes")
deseq5um = pyDESeq2.pyDESeq2(count_matrix=tccounts[tccounts.columns[np.array([0,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1,1],np.bool)]],
                         design_matrix=design_reg[np.array([0,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1],np.bool)],
                         design_formula="~DMSO - VHL",
                         gene_column="genes")
deseqn1um = pyDESeq2.pyDESeq2(count_matrix=tccounts[tccounts.columns[np.array([1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1],np.bool)]],
                         design_matrix=designoto[np.array([1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0],np.bool)],
                         design_formula="~DMSO - VHL",
                         gene_column="genes")

### NVP2 JQ1 MOLM13

In [ ]:
deseq1 = pyDESeq2.pyDESeq2(count_matrix = tccounts[tccounts.columns[np.array([1,1,1,1,1,1,0,0,0,0,0,0,0,0,1],np.bool)]],
                         design_matrix=design1[np.array([1,1,1,1,1,1,0,0,0,0,0,0,0,0],np.bool)],
                         design_formula="~DMSO - JQ1",
                         gene_column="genes")
deseq2 = pyDESeq2.pyDESeq2(count_matrix = tccounts[tccounts.columns[np.array([1,1,1,0,0,0,1,1,1,0,0,0,0,0,1],np.bool)]],
                         design_matrix=design2[np.array([1,1,1,0,0,0,1,1,1,0,0,0,0,0],np.bool)],
                         design_formula="~DMSO - NVP2low",
                         gene_column="genes")
deseq3 = pyDESeq2.pyDESeq2(count_matrix = tccounts[tccounts.columns[np.array([1,1,1,0,0,0,0,0,0,1,1,0,0,0,1],np.bool)]],
                         design_matrix=design3[np.array([1,1,1,0,0,0,0,0,0,1,1,0,0,0],np.bool)],
                         design_formula="~DMSO - NVP2high",
                         gene_column="genes")
deseq4 = pyDESeq2.pyDESeq2(count_matrix = tccounts[tccounts.columns[np.array([1,1,1,0,0,0,0,0,0,0,0,1,1,1,1],np.bool)]],
                         design_matrix=design4[np.array([1,1,1,0,0,0,0,0,0,0,0,1,1,1],np.bool)],
                         design_formula="~DMSO - NVP2lowJQ1",
                         gene_column="genes")

### K562 MV411 comparison

In [ ]:
deseqK562 = pyDESeq2.pyDESeq2(count_matrix=tccounts[tccounts.columns[np.array([1,1,1,1,1,1,0,0,0,0,0,0,1],np.bool)]],
                         design_matrix=designK562[np.array([1,1,1,1,1,1,0,0,0,0,0,0],np.bool)],
                         design_formula="~DMSO - JQ1",
                         gene_column="genes")
deseqMV411 = pyDESeq2.pyDESeq2(count_matrix=tccounts[tccounts.columns[np.array([0,0,0,0,0,0,1,1,1,1,1,1,1],np.bool)]],
                         design_matrix=designMV411[np.array([0,0,0,0,0,0,1,1,1,1,1,1],np.bool)],
                         design_formula="~DMSO - JQ1",
                         gene_column="genes")

### K562 HCT116 ccomparrison

In [ ]:
deseq_K562 = pyDESeq2.pyDESeq2(count_matrix=tccounts[tccounts.columns[np.array([1,1,1,1,1,1,0,0,0,0,0,0,1],np.bool)]],
                         design_matrix=design_K562[np.array([1,1,1,1,1,1,0,0,0,0,0,0],np.bool)],
                         design_formula="~DMSO - VHL",
                         gene_column="genes")
deseq_HCT116 = pyDESeq2.pyDESeq2(count_matrix=tccounts[tccounts.columns[np.array([0,0,0,0,0,0,1,1,1,1,1,1,1],np.bool)]],
                         design_matrix=design_HCT116[np.array([0,0,0,0,0,0,1,1,1,1,1,1],np.bool)],
                         design_formula="~DMSO - VHL",
                         gene_column="genes")

In [ ]:
deseq1.run_estimate_size_factors(geoMeans = np.exp(np.mean(np.log(
    readcounts[readcounts.columns[np.array([0,0,0,1,1,1,0,0,0,0,0,0,0,0,0], np.bool)]].values+1), 1)))
deseq2.run_estimate_size_factors(geoMeans = np.exp(np.mean(np.log(
    readcounts[readcounts.columns[np.array([0,0,0,0,0,0,1,1,1,0,0,0,0,0,0], np.bool)]].values+1), 1)))
deseq3.run_estimate_size_factors(geoMeans = np.exp(np.mean(np.log(
    readcounts[readcounts.columns[np.array([0,0,0,0,0,0,0,0,0,1,1,0,0,0,0], np.bool)]].values+1), 1)))
deseq4.run_estimate_size_factors(geoMeans = np.exp(np.mean(np.log(
    readcounts[readcounts.columns[np.array([0,0,0,0,0,0,0,0,0,0,0,1,1,1,0], np.bool)]].values+1), 1)))

In [ ]:
deseq_K562.run_estimate_size_factors(normMatrix = (readcounts[readcounts.columns[
        np.array([1,1,1,1,1,1,0,0,0,0,0,0,0],np.bool)]].values+1))
deseq_HCT116.run_estimate_size_factors(normMatrix = (readcounts[readcounts.columns[
        np.array([0,0,0,0,0,0,1,1,1,1,1,1,0],np.bool)]].values+1))

In [ ]:
deseq10um.run_estimate_size_factors(geoMeans = np.exp(np.mean(np.log(
    readcounts[readcounts.columns[
        np.array([1,0,0,0,1,1,1,0,1,1,1,0,0,1,0,0,0],np.bool)]].values+1), 1)))
deseq1um.run_estimate_size_factors(geoMeans = np.exp(np.mean(np.log(
    readcounts[readcounts.columns[
        np.array([0,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1,0],np.bool)]].values+1), 1)))
deseqn5um.run_estimate_size_factors(geoMeans = np.exp(np.mean(np.log(
    readcounts[readcounts.columns[
        np.array([1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0],np.bool)]].values+1), 1)))

In [ ]:
deseqK562.run_estimate_size_factors(geoMeans = np.exp(np.mean(np.log(
    readcounts[readcounts.columns[np.array([1,1,1,0,0,0,0,0,0,0,0,0,0],np.bool)]].values+1), 1)))
deseqMV411.run_estimate_size_factors(geoMeans = np.exp(np.mean(np.log(
    readcounts[readcounts.columns[np.array([0,0,0,0,0,0,1,1,1,0,0,0,0],np.bool)]].values+1), 1)))

In [ ]:
deseq10um.run_deseq()
deseq1um.run_deseq()
deseqn5um.run_deseq()
deseq10um.get_deseq_result()
deseq1um.get_deseq_result()
deseqn5um.get_deseq_result()
res10um = deseq10um.deseq_result
deseq1um = deseq1um.deseq_result
deseqn5um = deseqn5um.deseq_result
res10um.pvalue = np.nan_to_num(np.array(res10um.pvalue), 1)
res10um.log2FoldChange = np.nan_to_num(np.array(res10um.log2FoldChange), 0)
deseq1um.pvalue = np.nan_to_num(np.array(deseq1um.pvalue), 1)
deseq1um.log2FoldChange = np.nan_to_num(np.array(deseq1um.log2FoldChange), 0)
deseqn5um.pvalue = np.nan_to_num(np.array(deseqn5um.pvalue), 1)
deseqn5um.log2FoldChange = np.nan_to_num(np.array(deseqn5um.log2FoldChange), 0)
res10um.log2FoldChange = -res10um.log2FoldChange
deseq1um.log2FoldChange = -deseq1um.log2FoldChange
deseqn5um.log2FoldChange = -deseqn5um.log2FoldChange
res10um["gene_id"] = res10um.genes
deseq1um["gene_id"] = deseq1um.genes
deseqn5um["gene_id"] = deseqn5um.genes

In [ ]:
deseq_K562.run_deseq()
deseq_HCT116.run_deseq()
deseq_K562.get_deseq_result()
deseq_HCT116.get_deseq_result()
res_K562 = deseq_K562.deseq_result
res_HCT116 = deseq_HCT116.deseq_result
res_K562.pvalue = np.nan_to_num(np.array(res_K562.pvalue), 1)
res_K562.log2FoldChange = np.nan_to_num(np.array(res_K562.log2FoldChange), 0)
res_HCT116.pvalue = np.nan_to_num(np.array(res_HCT116.pvalue), 1)
res_HCT116.log2FoldChange = np.nan_to_num(np.array(res_HCT116.log2FoldChange), 0)
res_HCT116.log2FoldChange = -res_HCT116.log2FoldChange
res_K562.log2FoldChange = -res_K562.log2FoldChange
res_K562["gene_id"] = res_K562.genes
res_HCT116["gene_id"] = res_HCT116.genes

In [ ]:
deseq_K562.run_deseq()
deseq_MV411.run_deseq()
deseq_K562.get_deseq_result()
deseq_MV411.get_deseq_result()
res_K562 = deseq_K562.deseq_result
res_MV411 = deseq_MV411.deseq_result
res_K562.pvalue = np.nan_to_num(np.array(res_K562.pvalue), 1)
res_K562.log2FoldChange = np.nan_to_num(np.array(res_K562.log2FoldChange), 0)
res_MV411.pvalue = np.nan_to_num(np.array(res_MV411.pvalue), 1)
res_MV411.log2FoldChange = np.nan_to_num(np.array(res_MV411.log2FoldChange), 0)
deseq_MV411.log2FoldChange = -deseq_MV411.log2FoldChange
res_K562.log2FoldChange = -res_K562.log2FoldChange
res_K562["gene_id"] = res_K562.genes
deseq_MV411["gene_id"] = deseq_MV411.genes

In [ ]:
deseq1.run_deseq()
deseq2.run_deseq()
deseq3.run_deseq()
deseq4.run_deseq()
deseq2.get_deseq_result()
deseq1.get_deseq_result()
deseq3.get_deseq_result()
deseq4.get_deseq_result()
res1 = deseq1.deseq_result
res2 = deseq2.deseq_result
res3 = deseq3.deseq_result
res4 = deseq4.deseq_result
res1.pvalue = np.nan_to_num(np.array(res1.pvalue), 1)
res1.log2FoldChange = np.nan_to_num(np.array(res1.log2FoldChange), 0)
res2.pvalue = np.nan_to_num(np.array(res2.pvalue), 1)
res2.log2FoldChange = np.nan_to_num(np.array(res2.log2FoldChange), 0)
res3.pvalue = np.nan_to_num(np.array(res3.pvalue), 1)
res3.log2FoldChange = np.nan_to_num(np.array(res3.log2FoldChange), 0)
res4.pvalue = np.nan_to_num(np.array(res4.pvalue), 1)
res4.log2FoldChange = np.nan_to_num(np.array(res4.log2FoldChange), 0)
res1.log2FoldChange = -res1.log2FoldChange
res2.log2FoldChange = -res2.log2FoldChange
res3.log2FoldChange = -res3.log2FoldChange
res4.log2FoldChange = -res4.log2FoldChange
res1["gene_id"] = res1.genes
res2["gene_id"] = res2.genes
res3["gene_id"] = res3.genes
res4["gene_id"] = res4.genes

In [ ]:
res[res.genes=='MYC']

## PLOTS

### whisker's plots

In [ ]:
res = res1[res1.baseMean>10]
res.baseMean= np.log2(1+res.baseMean)
res["type"] = ['ctf' if i else "other" for i in  res.genes.isin(ctf)]
ax = sns.boxplot(data=res,x='type',y='log2FoldChange').set_title("JQ1")
ax.savefig('../results/'+project+"/plots/"+version+"_whiskersJQ1_MOLM13_logfch.pdf")

In [ ]:
ax =sns.boxplot(data=res,x='type',y='baseMean').set_title("JQ1")
ax.savefig('../results/'+project+"/plots/"+version+"_whiskers_NVP2low_MOLM13_baseMean.pdf")

In [ ]:
res = res2[res2.baseMean>10]
res.baseMean= np.log2(1+res.baseMean)
res["type"] = ['ctf' if i else "other" for i in  res.genes.isin(ctf)]
ax= sns.boxplot(data=res,x='type',y='log2FoldChange').set_title("NVP2low")
ax.savefig('../results/'+project+"/plots/"+version+"_whiskers_NVP2low_MOLM13_logfch.pdf")

In [ ]:
ax = sns.boxplot(data=res,x='type',y='baseMean').set_title("NVP2low")
ax.savefig('../results/'+project+"/plots/"+version+"_whiskers_NVP2low_baseMean.pdf")

In [ ]:
res = res3[res3.baseMean>10]
res.baseMean= np.log2(1+res.baseMean)
res["type"] = ['ctf' if i else "other" for i in  res.genes.isin(ctf)]
ax = sns.boxplot(data=res,x='type',y='log2FoldChange').set_title("NVP2high")
ax.savefig('../results/'+project+"/plots/"+version+"_whiskers_NVP2high_MOLM13_logfch.pdf")

In [ ]:
ax =sns.boxplot(data=res,x='type',y='baseMean').set_title("NVP2high")
ax.savefig('../results/'+project+"/plots/"+version+"_whiskers_NVP2low_MOLM13_baseMean.pdf")

In [ ]:
res = res4[res4.baseMean>10]
res.baseMean= np.log2(1+res.baseMean)
res["type"] = ['ctf' if i else "other" for i in  res.genes.isin(ctf)]
ax =sns.boxplot(data=res,x='type',y='log2FoldChange').set_title("NVP2low+JQ1")
ax.savefig('../results/'+project+"/plots/"+version+"_whiskers_NVP2lowJQ1_MOLM13_logfch.pdf")

In [ ]:
ax = sns.boxplot(data=res,x='type',y='baseMean').set_title("NVP2low+JQ1")
ax.savefig('../results/'+project+"/plots/"+version+"_whiskers_NVP2lowJQ1_MOLM13_baseMean.pdf")

#### other

In [ ]:
res = res_MV411[res_MV411.baseMean>10]
res.baseMean= np.log2(1+res.baseMean)
res["type"] = ['ctf' if i else "other" for i in  res.genes.isin(ctf)]
ax = sns.boxplot(data=res,x='type',y='log2FoldChange').set_title("MV411")
ax.savefig('../results/'+project+"/plots/"+version+"_whiskers_MV411_JQ1_logfch.pdf")

In [ ]:
ax =sns.boxplot(data=res,x='type',y='baseMean').set_title("MV411")
ax.savefig('../results/'+project+"/plots/"+version+"_whiskers_MV411_JQ1_baseMean.pdf")

In [ ]:
res = K562res[K562res.baseMean>10]
res.baseMean= np.log2(1+res.baseMean)
res["type"] = ['ctf' if i else "other" for i in  res.genes.isin(ctf)]
ax =sns.boxplot(data=res,x='type',y='log2FoldChange').set_title("K562")
ax.savefig('../results/'+project+"/plots/"+version+"_whiskers_JQ1_K562_logfch.pdf")

In [ ]:
ax =sns.boxplot(data=res,x='type',y='baseMean').set_title("K562")
ax.savefig('../results/'+project+"/plots/"+version+"_whiskers_JQ1_K562_baseMean.pdf")

## Scatter plot comparison

In [ ]:
mix = pd.DataFrame()
mix["gene_id"] = K562res["gene_id"]
mix['x'] = - MV411res.log2FoldChange
mix['y'] = K562res.log2FoldChange

In [ ]:
h.scatter(mix[['x','y']].values[:12000], 
               mix['gene_id'].values.tolist()[:12000], radi= 0.06, alpha=0.3,
              colors = [0 if i in ctf else 1 for i in mix['gene_id'].values.tolist()[:12000]])

## Regular volcanos

In [ ]:
res_K562.to_csv("../results/"+project+"/"+version+'_'+minvar_toremove+'_'+mincount_toremove+'_K562_JQ1_deseq.csv')
res_HCT116.to_csv("../results/"+project+"/"+version+'_'+minvar_toremove+'_'+mincount_toremove+'_HCT116_JQ1_deseq.csv')

In [ ]:
MV411res.to_csv("../results/"+project+"/"+version+'_'+minvar_toremove+'_'+mincount_toremove+'_MV411_JQ1_deseq.csv')
K562res.to_csv("../results/"+project+"/"+version+'_'+minvar_toremove+'_'+mincount_toremove+'_K562_JQ1_deseq.csv')

In [ ]:
res1.to_csv("../results/"+project+"/"+version+'_'+minvar_toremove+'_'+mincount_toremove+'_jq1_deseq.csv')
res2.to_csv("../results/"+project+"/"+version+'_'+minvar_toremove+'_'+mincount_toremove+'_nvmlow_deseq.csv')
res3.to_csv("../results/"+project+"/"+version+'_'+minvar_toremove+'_'+mincount_toremove+'_nvmhigh_deseq.csv')
res4.to_csv("../results/"+project+"/"+version+'_'+minvar_toremove+'_'+mincount_toremove+'_nvmlowjq1_deseq.csv')

In [ ]:
res10um.to_csv("../results/"+project+"/"+version+'_'+minvar_toremove+'_'+mincount_toremove+'_jq1_10um_deseq.csv')
res1um.to_csv("../results/"+project+"/"+version+'_'+minvar_toremove+'_'+mincount_toremove+'_jq1_1um_deseq.csv')
res5um.to_csv("../results/"+project+"/"+version+'_'+minvar_toremove+'_'+mincount_toremove+'_jq1_5um_deseq.csv')

we can conclude that we get similar results to the slamseq myc paper although it seems that our values are a bit skewed toward higher expression than what is on the slamseq paper. It mightt be explained by the pseudo count of 1 that I did not set. Because I think it would highly bias the DESeq algorithm.

In [ ]:
volcano(res,tohighlight=ctf, searchbox=True, title='DESeq results of MV411 under 10um of JQ1 in volcano plot', folder='../results/'+project+'/plots/'+version+'_'+minvar_toremove+'_'+mincount_toremove)
volcano(res_neg,tohighlight=ctf, searchbox=True, title='DESeq results of MV411 under 1um of JQ1 in volcano plot',folder='../results/'+project+'/plots/'+version+'_'+minvar_toremove+'_'+mincount_toremove)
volcano(resnoto,tohighlight=ctf, searchbox=True, title='DESeq results of MV411 under 5um of JQ1 in volcano plot',folder='../results/'+project+'/plots/'+version+'_'+minvar_toremove+'_'+mincount_toremove)

### NVP2 JQ1 MOLM13

In [ ]:
h.volcano(res1,tohighlight=ctf, searchbox=True, title="JQ1", folder='../results/'+project+'/plots/'+version+'_'+minvar_toremove+'_'+mincount_toremove)
h.volcano(res2,tohighlight=ctf, searchbox=True, title="NVP2low", folder='../results/'+project+'/plots/'+version+'_'+minvar_toremove+'_'+mincount_toremove)
h.volcano(res3,tohighlight=ctf, searchbox=True, title="NVP2hgih", folder='../results/'+project+'/plots/'+version+'_'+minvar_toremove+'_'+mincount_toremove)
h.volcano(res4,tohighlight=ctf, searchbox=True, title="NVP2low+JQ1", folder='../results/'+project+'/plots/'+version+'_'+minvar_toremove+'_'+mincount_toremove)

### K562 HCT116 comparison

In [ ]:
scaling6 = 

In [ ]:
res_K562.log2FoldChange = res_K562.log2FoldChange - scaling6

In [ ]:
volcano(res_HCT116,tohighlight=ctf, searchbox=True, minpval=4, minlogfold=0.3, title='HCT116 under MYC AID', folder='../results/'+project+'/plots/'+version+'_'+minvar_toremove+'_'+mincount_toremove)
volcano(res_K562,tohighlight=ctf, searchbox=True, maxvalue=20,minpval=3, minlogfold=0.4, title='K562 under MYC AID', folder='../results/'+project+'/plots/'+version+'_'+minvar_toremove+'_'+mincount_toremove)

### K562 MV411 comparison

In [ ]:
h.volcano(MV411res,tohighlight=ctf, searchbox=True, title="MV411 under JQ1", folder='../results/'+project+'/plots/'+version+'_'+minvar_toremove+'_'+mincount_toremove)
h.volcano(K562res,tohighlight=ctf, searchbox=True, title="K562 under JQ1", folder='../results/'+project+'/plots/'+version+'_'+minvar_toremove+'_'+mincount_toremove)